In [13]:
import pandas as pd
import numpy as np
import os
from string import punctuation
import gensim
import warnings
warnings.filterwarnings('ignore')
# from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,precision_score,recall_score

In [2]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
os.getcwd()

'/Users/manpreetsi/My Python Jupyter Notebook'

In [4]:
import gensim.downloader as api

In [5]:
wiki_embeddings = api.load('glove-wiki-gigaword-100')

In [6]:
wiki_embeddings['apple']

array([-0.5985   , -0.46321  ,  0.13001  , -0.019576 ,  0.4603   ,
       -0.3018   ,  0.8977   , -0.65634  ,  0.66858  , -0.49164  ,
        0.037557 , -0.050889 ,  0.6451   , -0.53882  , -0.3765   ,
       -0.04312  ,  0.51384  ,  0.17783  ,  0.28596  ,  0.92063  ,
       -0.49349  , -0.48583  ,  0.61321  ,  0.78211  ,  0.19254  ,
        0.91228  , -0.055596 , -0.12512  , -0.65688  ,  0.068557 ,
        0.55629  ,  1.611    , -0.0073642, -0.48879  ,  0.45493  ,
        0.96105  , -0.063369 ,  0.17432  ,  0.9814   , -1.3125   ,
       -0.15801  , -0.54301  , -0.13888  , -0.26146  , -0.3691   ,
        0.26844  , -0.24375  , -0.19484  ,  0.62583  , -0.7377   ,
        0.38351  , -0.75004  , -0.39053  ,  0.091498 , -0.36591  ,
       -1.4715   , -0.45228  ,  0.2256   ,  1.1412   , -0.38526  ,
       -0.06716  ,  0.57288  , -0.39191  ,  0.31302  , -0.29235  ,
       -0.96157  ,  0.15154  , -0.21659  ,  0.25103  ,  0.096967 ,
        0.2843   ,  1.4296   , -0.50565  , -0.51374  , -0.4721

In [7]:
wiki_embeddings['mango']

array([-1.2291  ,  0.10335 , -0.26987 ,  0.48226 ,  0.33361 ,  0.23705 ,
        0.63042 ,  0.43053 ,  0.5771  , -0.52647 , -0.60073 ,  0.23436 ,
        0.51035 ,  0.15328 ,  0.38902 ,  0.70331 ,  1.1031  ,  0.2827  ,
        0.65623 ,  1.0383  , -0.036059,  0.3269  ,  0.92031 ,  0.12273 ,
        0.48289 ,  1.7486  ,  0.57833 , -0.44843 , -0.36878 , -0.58252 ,
       -0.25886 ,  0.37798 ,  0.50073 , -0.24892 , -0.38156 ,  0.72429 ,
        0.18912 ,  0.35381 ,  0.76222 , -0.83072 ,  0.37245 , -0.74036 ,
       -0.093917, -0.19363 ,  0.76738 ,  0.25973 , -0.3613  ,  0.11266 ,
        0.074644, -0.50861 , -0.656   , -0.64469 , -0.26661 ,  0.33047 ,
       -0.47061 ,  0.10678 , -0.27091 ,  0.74263 , -0.021164, -0.036004,
       -0.40231 ,  0.79084 , -0.22085 ,  0.22727 , -0.18947 , -0.89136 ,
       -0.20937 , -0.83009 ,  0.431   , -0.30165 ,  0.045189,  0.79236 ,
       -0.067395, -0.62576 , -0.10222 ,  0.36835 , -0.29499 , -0.090253,
        0.1337  ,  0.77281 ,  0.68425 , -0.66911 , 

In [8]:
wiki_embeddings.similarity('canada','ontario')

0.63857234

In [9]:
wiki_embeddings.most_similar('computing')

[('computer', 0.7141730785369873),
 ('computational', 0.7098971009254456),
 ('software', 0.7074766159057617),
 ('technology', 0.7047896981239319),
 ('networking', 0.7002713680267334),
 ('desktop', 0.6805328726768494),
 ('automation', 0.6773476004600525),
 ('computation', 0.6757926344871521),
 ('applications', 0.6627570986747742),
 ('systems', 0.6609983444213867)]

### Import spam-ham data to build word2vec RandomForest Ml Model

In [5]:
data = pd.read_csv('SMSSpamCollection.tsv',header=None,sep='\t')

In [6]:

data.rename(columns={0:'target',1:'text'},inplace=True)

#### Gensim exsiting function to clean the data and tokenize it

In [7]:
data['clean_text']=data['text'].apply(lambda x : gensim.utils.simple_preprocess(x))

In [8]:
data.head()

,target,text,clean_text
0,ham,I've been searching for the right words to tha...,"[ve, been, searching, for, the, right, words, ..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, don, think, he, goes, to, usf, he, lives..."
3,ham,Even my brother is not like to speak with me. ...,"[even, my, brother, is, not, like, to, speak, ..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[have, date, on, sunday, with, will]"


In [9]:
data['target']=data['target'].apply(lambda x : 1 if x=='spam' else 0)

In [10]:
data.head(3)

,target,text,clean_text
0,0,I've been searching for the right words to tha...,"[ve, been, searching, for, the, right, words, ..."
1,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup..."
2,0,"Nah I don't think he goes to usf, he lives aro...","[nah, don, think, he, goes, to, usf, he, lives..."


In [14]:
x_train,x_test,y_train,y_test= train_test_split(data['clean_text'],
                                                data['target'],
                                                test_size=.2,
                                                stratify=data['target'])


In [15]:
x_train.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [16]:
y_train.value_counts()

target
0    3857
1     597
Name: count, dtype: int64

In [17]:
x_train.shape

(4454,)

In [18]:
x_train.head()

0    [what, is, this, hex, place, you, talk, of, ex...
1    [get, out, of, class, in, bsn, in, like, lt, g...
2    [sorry, ll, call, you, later, am, in, meeting,...
3    [when, people, see, my, msgs, they, think, iam...
4    [thank, you, winner, notified, by, sms, good, ...
Name: clean_text, dtype: object

In [19]:
print(x_train.iloc[0])

['what', 'is', 'this', 'hex', 'place', 'you', 'talk', 'of', 'explain']


In [20]:
print(len(x_train.iloc[0])) ## 9 words in 1st sentence of x_train

9


### Training Word2Vec model on our data x_train dataset

In [22]:
## input is a list of sentences (tokenized)

w2v_model= gensim.models.Word2Vec(sentences = x_train,min_count=2,vector_size=300,window=7,sg=0) 
# sg =0 means CBOW is used ; default sg=1

In [23]:
print(w2v_model)

Word2Vec<vocab=3387, vector_size=300, alpha=0.025>


In [25]:
len(w2v_model.wv.index_to_key) ## learned 3378 words

3387

In [29]:
w2v_model.wv.key_to_index['all']

46

In [32]:
w2v_model.wv[46]==w2v_model.wv['all']
# meaning, we can embedding by word or by index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [34]:
words = set(w2v_model.wv.key_to_index)

In [35]:
words

{'hw',
 'body',
 'super',
 'urself',
 'class',
 'channel',
 'dating',
 'covers',
 'end',
 'max',
 'sea',
 'inclusive',
 'hip',
 'comedy',
 'jacket',
 'personality',
 'networks',
 'decimal',
 'worth',
 'seven',
 'share',
 'likes',
 'side',
 'mega',
 'height',
 'denis',
 'table',
 'st',
 'children',
 'value',
 'drink',
 'taste',
 'blank',
 'boat',
 'hard',
 'bother',
 'infront',
 'these',
 'stuff',
 'lp',
 'broke',
 'cherish',
 'little',
 'gin',
 'driving',
 'salary',
 'wearing',
 'camcorder',
 'foot',
 'sory',
 'ques',
 'someone',
 'sec',
 'tsunamis',
 'welcome',
 'payment',
 'sha',
 'continue',
 'discreet',
 'pole',
 'slap',
 'knw',
 'wonders',
 'grahmbell',
 'jeans',
 'ideas',
 'greatest',
 'nxt',
 'blake',
 'sticky',
 'dey',
 'mail',
 'ahmad',
 'becomes',
 'prize',
 'accept',
 'group',
 'escape',
 'dirty',
 'woke',
 'roads',
 'expression',
 'pink',
 'missing',
 'doesnt',
 'stamped',
 'living',
 'beware',
 'shower',
 'fills',
 'sh',
 'ideal',
 'immediately',
 'research',
 'gbp',
 'bil

In [37]:
w2v_model.wv.key_to_index

{'to': 0,
 'you': 1,
 'the': 2,
 'and': 3,
 'in': 4,
 'is': 5,
 'me': 6,
 'it': 7,
 'my': 8,
 'for': 9,
 'your': 10,
 'of': 11,
 'call': 12,
 'have': 13,
 'that': 14,
 'on': 15,
 'now': 16,
 'are': 17,
 'can': 18,
 'but': 19,
 'so': 20,
 'not': 21,
 'or': 22,
 'ur': 23,
 'get': 24,
 'at': 25,
 'do': 26,
 'will': 27,
 'we': 28,
 'no': 29,
 'if': 30,
 'just': 31,
 'be': 32,
 'with': 33,
 'this': 34,
 'gt': 35,
 'lt': 36,
 'ok': 37,
 'up': 38,
 'how': 39,
 'when': 40,
 'free': 41,
 'what': 42,
 'from': 43,
 'go': 44,
 'll': 45,
 'all': 46,
 'out': 47,
 'know': 48,
 'good': 49,
 'got': 50,
 'like': 51,
 'then': 52,
 'day': 53,
 'its': 54,
 'there': 55,
 'he': 56,
 'am': 57,
 'come': 58,
 'was': 59,
 'only': 60,
 'love': 61,
 'time': 62,
 'send': 63,
 'want': 64,
 'text': 65,
 'txt': 66,
 'going': 67,
 'stop': 68,
 'don': 69,
 'by': 70,
 'lor': 71,
 'need': 72,
 'one': 73,
 'as': 74,
 'she': 75,
 'sorry': 76,
 'home': 77,
 'about': 78,
 'see': 79,
 'back': 80,
 'tell': 81,
 'today': 82,
 'h

In [36]:
w2v_model.wv.index_to_key

['to',
 'you',
 'the',
 'and',
 'in',
 'is',
 'me',
 'it',
 'my',
 'for',
 'your',
 'of',
 'call',
 'have',
 'that',
 'on',
 'now',
 'are',
 'can',
 'but',
 'so',
 'not',
 'or',
 'ur',
 'get',
 'at',
 'do',
 'will',
 'we',
 'no',
 'if',
 'just',
 'be',
 'with',
 'this',
 'gt',
 'lt',
 'ok',
 'up',
 'how',
 'when',
 'free',
 'what',
 'from',
 'go',
 'll',
 'all',
 'out',
 'know',
 'good',
 'got',
 'like',
 'then',
 'day',
 'its',
 'there',
 'he',
 'am',
 'come',
 'was',
 'only',
 'love',
 'time',
 'send',
 'want',
 'text',
 'txt',
 'going',
 'stop',
 'don',
 'by',
 'lor',
 'need',
 'one',
 'as',
 'she',
 'sorry',
 'home',
 'about',
 'see',
 'back',
 'tell',
 'today',
 'hi',
 'still',
 'da',
 'dont',
 'reply',
 'our',
 'later',
 'mobile',
 'new',
 'take',
 'please',
 'been',
 'her',
 'they',
 'phone',
 'any',
 'think',
 'some',
 'week',
 'dear',
 'who',
 'here',
 'did',
 'hope',
 'night',
 'has',
 'msg',
 'where',
 'pls',
 'well',
 'great',
 'him',
 'an',
 'claim',
 'give',
 'oh',
 'wat'

In [40]:
w2v_model.wv.similar_by_word('all')

[('of', 0.9999424815177917),
 ('so', 0.9999351501464844),
 ('but', 0.999929666519165),
 ('got', 0.9999285340309143),
 ('too', 0.9999279975891113),
 ('the', 0.999927282333374),
 ('some', 0.9999228119850159),
 ('in', 0.9999223947525024),
 ('they', 0.9999221563339233),
 ('one', 0.9999216198921204)]

In [41]:
w2v_model.wv.similar_by_word(46)


[('of', 0.9999424815177917),
 ('so', 0.9999351501464844),
 ('but', 0.999929666519165),
 ('got', 0.9999285340309143),
 ('too', 0.9999279975891113),
 ('the', 0.999927282333374),
 ('some', 0.9999228119850159),
 ('in', 0.9999223947525024),
 ('they', 0.9999221563339233),
 ('one', 0.9999216198921204)]

In [38]:
# error when the word is OOV.
# by defualt, word2vec can't handle OOV words
w2v_model.wv.similar_by_word('098ie5r4thuiefbg') ## words similar to 'too' from our corpus. 

KeyError: "Key '098ie5r4thuiefbg' not present in vocabulary"

In [148]:
w2v_model.wv['well'] ## vector representation of 1st word in 1st sentence 

array([-0.47207382, -0.13961108,  0.27765065, -0.25325432,  0.14577982,
        0.35287786,  0.2802804 ,  0.08277294, -0.18996426, -0.15296431,
       -0.38923594, -0.09188627,  0.24426636, -0.00168727, -0.08301775,
       -0.12685046,  0.08343118,  0.19974256,  0.09419371, -0.02815917,
       -0.00897433,  0.08922294, -0.13551353, -0.15928565,  0.05580015,
        0.37857947, -0.08890714, -0.16501957,  0.16081181, -0.10731183,
       -0.33358616, -0.16204563,  0.00571158, -0.23911788, -0.04284785,
        0.08892088, -0.00520516, -0.05372594,  0.37675884,  0.203797  ,
       -0.17920554, -0.2007101 , -0.33767042,  0.27864558,  0.13575162,
       -0.41893944,  0.09700918, -0.34902534, -0.05929133,  0.4353584 ,
       -0.14796029,  0.49525717,  0.07391249, -0.11671714, -0.32324737,
        0.00063345,  0.40168667,  0.32498425, -0.09303671,  0.22008744,
        0.25034335,  0.06815549,  0.06212088, -0.12369293, -0.2962859 ,
        0.454941  , -0.12425042,  0.0013265 ,  0.1356045 ,  0.04

In [150]:
np.mean(w2v_model.wv['well']) ## mean value of word 'well' vector

0.0033620102

### converting each word in x_train and x_test to its vector representation learned from w2v_model

In [151]:
## x_train

x_train_vect = np.array([ np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in x_train])

In [152]:
x_train_vect.size ## This is equal to number of sentences (rows) in x_train


4454

In [153]:
## x_test

x_test_vect = np.array([ np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in x_test])

In [154]:
x_test_vect.size ## This is equal to number of sentences (rows) in x_test

1114

In [155]:
len(x_train_vect[0]) ## Model learnt 7 words out of 9 words in 1st sentence of x_train

7

In [157]:
x_train_vect[0][0] ## Vector representation of 1st word 'well'

array([-0.47207382, -0.13961108,  0.27765065, -0.25325432,  0.14577982,
        0.35287786,  0.2802804 ,  0.08277294, -0.18996426, -0.15296431,
       -0.38923594, -0.09188627,  0.24426636, -0.00168727, -0.08301775,
       -0.12685046,  0.08343118,  0.19974256,  0.09419371, -0.02815917,
       -0.00897433,  0.08922294, -0.13551353, -0.15928565,  0.05580015,
        0.37857947, -0.08890714, -0.16501957,  0.16081181, -0.10731183,
       -0.33358616, -0.16204563,  0.00571158, -0.23911788, -0.04284785,
        0.08892088, -0.00520516, -0.05372594,  0.37675884,  0.203797  ,
       -0.17920554, -0.2007101 , -0.33767042,  0.27864558,  0.13575162,
       -0.41893944,  0.09700918, -0.34902534, -0.05929133,  0.4353584 ,
       -0.14796029,  0.49525717,  0.07391249, -0.11671714, -0.32324737,
        0.00063345,  0.40168667,  0.32498425, -0.09303671,  0.22008744,
        0.25034335,  0.06815549,  0.06212088, -0.12369293, -0.2962859 ,
        0.454941  , -0.12425042,  0.0013265 ,  0.1356045 ,  0.04

## Average of all the vectors in A is B

### Vector representation of each word in 1st sentence in x_train  --- A

In [158]:
x_train_vect[0] 

array([[-0.47207382, -0.13961108,  0.27765065, ..., -0.20821226,
         0.2062172 ,  0.32293993],
       [-0.5827319 , -0.17569594,  0.34360495, ..., -0.25637046,
         0.25724363,  0.40173388],
       [-0.31146488, -0.09337773,  0.18458582, ..., -0.13767523,
         0.135458  ,  0.21555294],
       ...,
       [-0.15100725, -0.04451773,  0.08841141, ..., -0.06793901,
         0.06764171,  0.10566685],
       [-0.05761864, -0.01659412,  0.03306207, ..., -0.0266624 ,
         0.02599317,  0.04070001],
       [-0.3197871 , -0.09415666,  0.18703167, ..., -0.13980697,
         0.13952878,  0.21894592]], dtype=float32)

### Average vector representation of 1st sentence of x_train ---B

In [159]:
x_train_vect[0].mean(axis=0) 

array([-0.28544715, -0.08514708,  0.16795339, -0.15310659,  0.08834615,
        0.2134764 ,  0.17101133,  0.04903498, -0.11560898, -0.09197844,
       -0.23734532, -0.05588515,  0.14779124, -0.00102337, -0.04979344,
       -0.07829582,  0.05250305,  0.11996113,  0.05710137, -0.01699188,
       -0.00598269,  0.05468474, -0.08180215, -0.09657615,  0.03460512,
        0.22984275, -0.05466522, -0.10065023,  0.09735962, -0.06497046,
       -0.20189078, -0.09746213,  0.00378774, -0.14462355, -0.02626701,
        0.0543437 , -0.00391591, -0.03136469,  0.22855099,  0.12303463,
       -0.10895541, -0.12110186, -0.20507777,  0.16906823,  0.08255047,
       -0.25452536,  0.05845978, -0.212147  , -0.0361036 ,  0.26433584,
       -0.09003551,  0.30009618,  0.0445773 , -0.07019343, -0.19632722,
        0.00059195,  0.24318512,  0.19674762, -0.0556645 ,  0.13335897,
        0.15165234,  0.04123702,  0.03756768, -0.0740245 , -0.17953202,
        0.27620628, -0.0754468 ,  0.00082327,  0.08181147,  0.02

In [166]:
## Repeating the above for all the sentences x_train
x_train_vect_avg=[]
for v in x_train_vect:
    if v.size:
        x_train_vect_avg.append(v.mean(axis=0))
        
    else :
        x_train_vect_avg.append(np.zeros(300,dtype=float)) ## 300 as our vector size is 300

In [167]:
## Similarly for x_test

x_test_vect_avg=[]
for v in x_test_vect:
    if v.size:
        x_test_vect_avg.append(v.mean(axis=0))
        
    else :
        x_test_vect_avg.append(np.zeros(300,dtype=float)) ## 300 as our vector size is 300

In [162]:
x_train_vect_avg[0]

array([-0.28544715, -0.08514708,  0.16795339, -0.15310659,  0.08834615,
        0.2134764 ,  0.17101133,  0.04903498, -0.11560898, -0.09197844,
       -0.23734532, -0.05588515,  0.14779124, -0.00102337, -0.04979344,
       -0.07829582,  0.05250305,  0.11996113,  0.05710137, -0.01699188,
       -0.00598269,  0.05468474, -0.08180215, -0.09657615,  0.03460512,
        0.22984275, -0.05466522, -0.10065023,  0.09735962, -0.06497046,
       -0.20189078, -0.09746213,  0.00378774, -0.14462355, -0.02626701,
        0.0543437 , -0.00391591, -0.03136469,  0.22855099,  0.12303463,
       -0.10895541, -0.12110186, -0.20507777,  0.16906823,  0.08255047,
       -0.25452536,  0.05845978, -0.212147  , -0.0361036 ,  0.26433584,
       -0.09003551,  0.30009618,  0.0445773 , -0.07019343, -0.19632722,
        0.00059195,  0.24318512,  0.19674762, -0.0556645 ,  0.13335897,
        0.15165234,  0.04123702,  0.03756768, -0.0740245 , -0.17953202,
        0.27620628, -0.0754468 ,  0.00082327,  0.08181147,  0.02

In [168]:
pd.DataFrame(x_train_vect_avg)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.285447,-0.085147,0.167953,-0.153107,0.088346,0.213476,0.171011,0.049035,-0.115609,-0.091978,...,-0.108804,0.037920,0.062469,0.052464,0.081657,-0.045964,-0.182975,-0.126140,0.125266,0.196682
1,-0.454783,-0.137820,0.266681,-0.244636,0.142640,0.342708,0.271581,0.077881,-0.184442,-0.146046,...,-0.173348,0.059022,0.101876,0.083718,0.129473,-0.071964,-0.290660,-0.203664,0.201978,0.313183
2,-0.555876,-0.167705,0.325787,-0.298167,0.174247,0.418107,0.332186,0.096436,-0.224113,-0.178390,...,-0.212661,0.073758,0.121491,0.102570,0.157233,-0.087633,-0.353896,-0.247947,0.244861,0.382215
3,-0.453674,-0.136456,0.266455,-0.243204,0.141939,0.341049,0.271295,0.078219,-0.184077,-0.145612,...,-0.173513,0.059718,0.099020,0.083234,0.128707,-0.071806,-0.289492,-0.202279,0.200115,0.311621
4,-0.486376,-0.146208,0.284847,-0.260903,0.151723,0.364885,0.289858,0.084232,-0.196398,-0.156451,...,-0.186302,0.064367,0.106321,0.089678,0.138068,-0.077314,-0.310168,-0.216781,0.214579,0.334667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,-0.464023,-0.140099,0.271963,-0.248990,0.145639,0.350104,0.277118,0.080219,-0.188928,-0.150591,...,-0.177963,0.060818,0.102707,0.085062,0.132376,-0.074056,-0.296907,-0.207500,0.206143,0.318532
4450,-0.428814,-0.129200,0.252255,-0.229954,0.134660,0.322350,0.256012,0.074051,-0.173004,-0.137952,...,-0.164002,0.056816,0.093425,0.079061,0.121503,-0.067749,-0.273286,-0.190995,0.189757,0.293766
4451,-0.474483,-0.142924,0.278490,-0.254528,0.148612,0.357215,0.283854,0.082219,-0.192106,-0.153081,...,-0.181031,0.063041,0.103879,0.086834,0.135042,-0.074759,-0.302454,-0.211506,0.209002,0.326035
4452,-0.402770,-0.121327,0.236315,-0.216030,0.125794,0.303071,0.240351,0.069884,-0.162530,-0.129857,...,-0.155332,0.052955,0.088889,0.074399,0.114195,-0.064327,-0.257389,-0.178937,0.177597,0.277044


In [107]:
len(x_train_vect_avg)

4454

In [108]:
len(y_train)

4454

In [109]:
len(x_test_vect_avg)

1114

In [110]:
len(y_test)

1114

## Random Forest Model

In [169]:
rf = RandomForestClassifier(n_estimators=500,max_depth=100,n_jobs=-1)

In [170]:
rf_model = rf.fit(x_train_vect_avg,y_train.values.ravel())

In [171]:
pred_proba = rf_model.predict_proba(x_test_vect_avg)[:,1]

In [172]:
pred = rf_model.predict(x_test_vect_avg)

In [173]:
roc_auc_score(y_test,pred_proba) 

0.9663594950794588

In [174]:
precision_score(y_test,pred)

0.8992248062015504

In [175]:
recall_score(y_test,pred)

0.7785234899328859